In [1]:
import numpy as np
import scipy
import utils
from time import time
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [2]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [3]:
print(Qt @ A @ P)

[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


## Phase-II

In [4]:
m = n = 500
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

In [5]:
B, _, _ = SVD.svd_phaseI(A)

In [6]:
numpy_mul_begin = time()
for i in range(2000):
    A@B
numpy_mul_end = time()
numpy_mul_end - numpy_mul_begin

4.359565258026123

In [7]:
fastMul_begin = time()
for i in range(2000):
    SVD.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
fastMul_end - fastMul_begin

5.630804061889648

**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [8]:
m = 1024
n = 1024
A = np.random.rand(m,n)

In [15]:
U, S, Vt = SVD.svd(A, phaseII='A')
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

phaseI: 11.5287s
phaseII: 58.4604s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


**Accuracy Test:**

In [10]:
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


**Scipy SVD**

In [11]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


## Test for p2

In [95]:
n=500
k = 5
A = np.array(np.diag([1/2]*n, 0)+np.diag([3/8]*(n-1), 1)+np.diag([1/8]*(n-1), -1))
print(A[5].sum())

print(np.prod(A.diagonal()))
A = np.linalg.matrix_power(A, k)
B, Qt, P = SVD.svd_phaseI(A)
U, T, Vt = SVD.svd_phaseII(B, Qt, P, phaseII="B1", eigen=QR.eigh_of_BBT)
# U, T, Vt = SVD.svd(A, phaseII="B1")
print(T)

1.0
3.054936363499605e-151
[1.8051e-03 1.7101e-03 1.6185e-03 1.5301e-03 1.4450e-03 1.3630e-03
 1.2841e-03 1.2082e-03 1.1354e-03 1.0654e-03 9.9823e-04 9.3386e-04
 8.7223e-04 8.1325e-04 7.5689e-04 7.0307e-04 6.5174e-04 6.0285e-04
 5.5634e-04 5.1215e-04 4.7023e-04 4.3053e-04 3.9299e-04 3.5756e-04
 3.2419e-04 2.9282e-04 2.6340e-04 2.3588e-04 2.1021e-04 1.8633e-04
 1.6419e-04 1.4374e-04 1.2493e-04 1.0770e-04 9.2005e-05 7.7786e-05
 6.4989e-05 5.3559e-05 4.3439e-05 3.4572e-05 2.6896e-05 2.0351e-05
 1.4872e-05 1.0393e-05 6.8405e-06 4.1415e-06 2.2224e-06 1.0023e-06
 3.5212e-07]
